In [ ]:
import talib
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

In [ ]:
class PowerXStrategy(Strategy):
    stop_loss_multiple = 1.5
    profit_target_multiple = 3.0

    def init(self):
        self.rsi = self.I(talib.RSI, self.data.Close, timeperiod=7)
        self.stoch_k = self.I(talib.STOCH, self.data.High, self.data.Low, self.data.Close, fastk_period=14, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)[0]
        self.macd_hist = self.I(talib.MACD, self.data.Close, fastperiod=12, slowperiod=26, signalperiod=9)[2]
        self.adr = self.I(talib.SMA, self.data.High, timeperiod=7) - self.I(talib.SMA, self.data.Low, timeperiod=7)

    def next(self):
        green_bar = (self.rsi[-1] > 50) & (self.stoch_k[-1] > 50) & (self.macd_hist[-1] > 0)
        red_bar = (self.rsi[-1] < 50) & (self.stoch_k[-1] < 50) & (self.macd_hist[-1] < 0)

        # ロング注文のためのストップロスと利益目標の計算
        long_profit_target = self.data.High[-1] + (self.profit_target_multiple * self.adr[-1])
        long_stop_loss = long_profit_target - (self.stop_loss_multiple * self.adr[-1])

        # ショート注文のためのストップロスと利益目標の計算
        short_profit_target = self.data.Low[-1] - (self.profit_target_multiple * self.adr[-1])
        short_stop_loss = short_profit_target + (self.stop_loss_multiple * self.adr[-1])

        # 注文の実行
        if green_bar and not self.position:
            self.buy(sl=long_stop_loss, tp=long_profit_target)
        elif red_bar and not self.position:
            self.sell(sl=short_stop_loss, tp=short_profit_target)



In [ ]:
# DataFrameがdfという変数に格納されているとします
df = pd.read_pickle('./data/df_rates_3H.pkl')
# 列名のリネーム
df.rename(columns={
    'time': 'Time',
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'tick_volume': 'TickVolume',
    'spread': 'Spread',
    'real_volume': 'RealVolume'
}, inplace=True)
df.info()

In [ ]:
# バックテストの実行
bt = Backtest(df, PowerXStrategy, cash=10000, commission=.002)
stats = bt.run()
print(stats)

# 結果のプロット
bt.plot()